In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np

In [3]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://10.22.112.39:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['10.22.112.39:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [4]:
from src.utils.CacheAdapter import JSONAdapter, EXP_END_OF_DATA
from src.utils.DatasetManager import ProjectsDatasetManager

In [5]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [6]:
adapter = JSONAdapter()
manager = ProjectsDatasetManager(20, cacheAdapter = adapter)

In [7]:
# using adapter to load data from the cache files

class DataFeeder(ProjectsDatasetManager):
    def __init__(self):
        pass


def getDataFromCache(batchSize = 32):
    cacheFileName = "cache__28-03-2025__(sufficient)_{index}.json"
    tempStorage = []    

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                yield tempStorage[:batchSize]
                tempStorage = tempStorage[batchSize:]

            manager.cacheAdapter.collectionName = cacheFileName.format(index = i)
            data = flatternData(manager.fromCache())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    yield tempStorage

In [8]:
def extractScannedUsers(data):
    return list(data.keys())


def getIgnored(batchSize = 32):
    cacheFileName = "cache__28-03-2025__(sufficient)_{index}.json"
    ignore = []

    i = 0
    while True:
        try:
            manager.cacheAdapter.collectionName = cacheFileName.format(index = i)
            data = extractScannedUsers(manager.fromCache())
            ignore.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    return ignore


In [12]:
ignore = getIgnored()
len(set(ignore))

cache__28-03-2025__(sufficient)_0.json
cache__28-03-2025__(sufficient)_1.json
cache__28-03-2025__(sufficient)_2.json
cache__28-03-2025__(sufficient)_3.json
cache__28-03-2025__(sufficient)_4.json
cache__28-03-2025__(sufficient)_5.json
cache__28-03-2025__(sufficient)_6.json
cache__28-03-2025__(sufficient)_7.json
cache__28-03-2025__(sufficient)_8.json
cache__28-03-2025__(sufficient)_9.json
cache__28-03-2025__(sufficient)_10.json
cache__28-03-2025__(sufficient)_11.json
cache__28-03-2025__(sufficient)_12.json
cache__28-03-2025__(sufficient)_13.json
cache__28-03-2025__(sufficient)_14.json
cache__28-03-2025__(sufficient)_15.json
cache__28-03-2025__(sufficient)_16.json
cache__28-03-2025__(sufficient)_17.json
cache__28-03-2025__(sufficient)_18.json
cache__28-03-2025__(sufficient)_19.json
cache__28-03-2025__(sufficient)_20.json
cache__28-03-2025__(sufficient)_21.json
cache__28-03-2025__(sufficient)_22.json
cache__28-03-2025__(sufficient)_23.json
cache__28-03-2025__(sufficient)_24.json
cache__28-

1192

In [10]:
2/0
g = getDataFromCache(batchSize = 2)
lst = []
for i in range(4):
    a = next(g)
    lst.append(a)

ZeroDivisionError: division by zero

In [ ]:
lst
#with open("/home/trukhinmaksim/src/data/cache/cache__28-03-2025__(sufficient)_20.json", encoding = "utf-8") as file:
#    print(file.readline()[:10])